In [1]:
import sys
import re
import time
import datetime
# Requires for ipython to pick up on twitter?
sys.path.append('/Library/Python/2.7/site-packages/')
import twitter
import pandas as pd
import func
# import pyowm # Historical API is paid

# inline plot
%matplotlib inline

/Users/albarron/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
#%load 'data/raw-twt2016-01-26-14/21/09.csv'
truth = pd.read_csv("data/truth_tweets.csv",sep=',',error_bad_lines=False)
twts = pd.read_csv("data/formated_twts.csv",sep=',',error_bad_lines=False)
weather = pd.read_csv("data/weather-add-twt2016-03-06-21:26:39.csv",sep='\t',error_bad_lines=False)

In [3]:
del truth['Unnamed: 0']
truth.head()

,id,text,train_id,is_delay,delay_minor,delay_med,delay_major,delay_catastrophic,is_backlog,is_canceled,is_passing,is_accident,is_medical,is_mechanical,is_customer,is_event
0,691768068385718275,#NB155 is 22 mins late. #Caltrain,155,1,0,0,1,0,0,0,0,0,0,0,0,0
1,690667494906814464,#NB 151 is running 10 mins behind #Caltrain,151,1,1,0,0,0,0,0,0,0,0,0,0,0
2,689936168893329408,#NB151 is 10 mins down at San Mateo. #Caltrain,151,1,1,0,0,0,0,0,0,0,0,0,0,0
3,689509733640732672,#SB138 will board on the northbound platform a...,138,0,0,0,0,0,0,0,0,0,0,0,0,0
4,689485484838416388,#SB216 delayed 13 minutes at SAT\n#NB225 delay...,216,1,0,1,0,0,0,0,0,0,0,0,0,0


In [4]:
del twts['Unnamed: 0']
twts.head(3)

,created_at,favorite_count,hashtags,id,in_reply_to_screen_name,retweet_count,text,day_of_week,day_of_month,month,time_of_day,topic_train,t_bullet,t_limited,t_northbound,timestamp,train_id,tweet_id
0,2016-01-25 23:42:14,1,"NB155, Caltrain",691768068385718275,NaN,2,#NB155 is 22 mins late. #Caltrain,0,25,1,error,'155',0,0,1,2016-01-25 23:42:14,155,691768068385718275
1,2016-01-22 22:48:57,NaN,"NB, Caltrain",690667494906814464,NaN,2,#NB 151 is running 10 mins behind #Caltrain,4,22,1,error,'151',0,0,1,2016-01-22 22:48:57,151,690667494906814464
2,2016-01-20 22:22:55,1,"NB151, Caltrain",689936168893329408,NaN,2,#NB151 is 10 mins down at San Mateo. #Caltrain,2,20,1,error,'151',0,0,1,2016-01-20 22:22:55,151,689936168893329408


In [5]:
wt = weather[['id','temp','precipiation','visability','windspeed','humidity','cloudcover']]

In [6]:
df = twts.merge(truth,on=['id','train_id'],how='inner')
df.shape

(583, 32)

In [14]:
df.head(2)
len(df['id'].unique())

475

In [9]:
df = df.merge(wt,on='id',how='left')
df.shape

(1048, 38)

In [ ]:
df.tail(4)

In [ ]:
filename = "./data/weather-add-twt{date}.csv".format(date=datetime.datetime.now().strftime("%Y-%m-%d-%H:%M:%S"))
# merged.to_csv(filename, sep='\t', encoding='utf-8')